In [ ]:
# pip install PyYAML

In [1]:
import os
import json
import shutil
from pathlib import Path
import yaml
import random

# 1. 경로 설정
origin_root = Path("origin_sample_dataset") # <-여기에 원본데이터셋 경로설정
yolo_root = Path("YOLOv11Dataset")  # <- 여기에 Yolo v11 변환데이터셋 경로설정 
yolo_root.mkdir(parents=True, exist_ok=True)

# 2. YOLO 디렉토리 구조 생성 (test 추가)
dirs = [
    "images/train",
    "images/val",
    "images/test",  # test 디렉토리 추가
    "labels/train",
    "labels/val",
    "labels/test"   # test 디렉토리 추가
]

for d in dirs:
    (yolo_root / d).mkdir(parents=True, exist_ok=True)

# 3. 클래스 매핑 정보 생성
class_mapping = {
    ("배", "정상"): 0,
    ("배", "질병"): 1,
    ("사과", "정상"): 2,
    ("사과", "질병"): 3
}

# 4. JSON → YOLO 레이블 변환 함수
def convert_label(json_path, img_width, img_height):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 클래스 결정
    crop = "배" if data['annotations']['crop'] == 1 else "사과"
    disease = "정상" if data['annotations']['disease'] == 0 else "질병"
    class_id = class_mapping[(crop, disease)]
    
    # 바운딩 박스 처리
    bbox_lines = []
    for point in data['annotations']['points']:
        xtl = point['xtl']
        ytl = point['ytl']
        xbr = point['xbr']
        ybr = point['ybr']
        
        # YOLO 형식으로 정규화
        x_center = ((xtl + xbr) / 2) / img_width
        y_center = ((ytl + ybr) / 2) / img_height
        width = (xbr - xtl) / img_width
        height = (ybr - ytl) / img_height
        
        bbox_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    return "\n".join(bbox_lines)

# 5. 데이터 처리 함수 (수정됨)
def process_dataset(src_type, target_types):
    print(f"\nProcessing {src_type} data...")
    
    # 원천 데이터 폴더 탐색
    for folder in (origin_root / src_type).iterdir():
        if folder.name.startswith("[원천]"):
            # 폴더명에서 작물과 상태 추출
            parts = folder.name[4:].split('_')
            crop = parts[0]
            status = parts[1].split('.')[1]
            
            print(f"  Processing images: {crop}-{status}")
            
            # 이미지 파일 목록 가져오기
            img_files = list(folder.glob("*.jpg"))
            random.shuffle(img_files)  # 무작위 섞기
            
            # 훈련 데이터는 전체를 사용
            if src_type == "Training":
                target_type = "train"
                for img_file in img_files:
                    process_image(img_file, folder, target_type)
            # 검증 데이터는 5:5로 분할
            elif src_type == "Validation":
                split_idx = len(img_files) // 2
                val_files = img_files[:split_idx]
                test_files = img_files[split_idx:]
                
                for img_file in val_files:
                    process_image(img_file, folder, "val")
                
                for img_file in test_files:
                    process_image(img_file, folder, "test")

# 5-1. 개별 이미지 처리 함수
def process_image(img_file, origin_folder, target_type):
    # 이미지 복사
    dest_dir = yolo_root / "images" / target_type
    shutil.copy(img_file, dest_dir / img_file.name)
    
    # 대응되는 JSON 파일 찾기
    json_path = None
    label_folder = origin_folder.name.replace("[원천]", "[라벨]")
    possible_json = [
        origin_root / origin_folder.parent.name / label_folder / f"{img_file.stem}.json",
        origin_root / origin_folder.parent.name / label_folder / f"{img_file.name}.json"
    ]
    
    for p in possible_json:
        if p.exists():
            json_path = p
            break
    
    if not json_path:
        print(f"    ! JSON not found for {img_file.name}")
        return
    
    # JSON에서 이미지 크기 추출
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    img_width = json_data['description']['width']
    img_height = json_data['description']['height']
    
    # 레이블 변환
    yolo_label = convert_label(json_path, img_width, img_height)
    
    # 레이블 저장 (YOLO 형식)
    label_dir = yolo_root / "labels" / target_type
    txt_path = label_dir / f"{img_file.stem}.txt"
    
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(yolo_label)

# 6. 데이터 처리 실행 (수정됨)
process_dataset("Training", ["train"])  # 훈련 데이터는 모두 train으로
process_dataset("Validation", ["val", "test"])  # 검증 데이터는 val과 test로 분할

# 7. dataset.yaml 생성 (test 추가)
yaml_content = {
    'path': str(yolo_root.resolve()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',  # test 경로 추가
    'names': {
        0: 'pear_normal',
        1: 'pear_disease',
        2: 'apple_normal',
        3: 'apple_disease'
    }
}

with open(yolo_root / "dataset.yaml", 'w', encoding='utf-8') as f:
    yaml.dump(yaml_content, f, allow_unicode=True, sort_keys=False)

# 8. 분할 결과 통계 출력
def print_stats():
    print("\nDataset split statistics:")
    for split in ['train', 'val', 'test']:
        img_count = len(list((yolo_root / "images" / split).glob("*.jpg")))
        label_count = len(list((yolo_root / "labels" / split).glob("*.txt")))
        print(f"  {split}: {img_count} images, {label_count} labels")

print("\nDataset conversion completed successfully!")
print(f"YOLO dataset structure created at: {yolo_root}")
print(f"Classes mapping: {yaml_content['names']}")
print_stats()


Processing Training data...
  Processing images: 배-정상
  Processing images: 배-질병
  Processing images: 사과-정상
  Processing images: 사과-질병

Processing Validation data...
  Processing images: 배-정상
  Processing images: 배-질병
  Processing images: 사과-정상
    ! JSON not found for V006_80_0_00_02_01_25_0_a01_20201028_0028_S01_1.jpg
    ! JSON not found for V006_80_0_00_02_01_25_0_a01_20201028_0010_S01.jpg
    ! JSON not found for V006_80_0_00_02_01_25_0_a01_20201028_0011_S01.jpg
    ! JSON not found for V006_80_0_00_02_01_25_0_a01_20201028_0016_S01_1.jpg
    ! JSON not found for V006_80_0_00_02_01_25_0_a01_20201019_0000_S01.jpg
  Processing images: 사과-질병

Dataset conversion completed successfully!
YOLO dataset structure created at: YOLOv11Dataset
Classes mapping: {0: 'pear_normal', 1: 'pear_disease', 2: 'apple_normal', 3: 'apple_disease'}

Dataset split statistics:
  train: 20 images, 20 labels
  val: 8 images, 6 labels
  test: 12 images, 9 labels
